In [22]:
import d6tflow
import pandas as pd

from d6tflow.functional import Flow
flow = Flow()

## Define workflow

In [23]:
@flow.step(d6tflow.tasks.TaskCache)
def get_data1(task):
    df = pd.DataFrame({'a':range(3)})
    task.save(df)

In [24]:
@flow.step(d6tflow.tasks.TaskCache)
def get_data2(task):
    df = pd.DataFrame({'a':range(3)})
    task.save(df)

In [25]:
@flow.step(d6tflow.tasks.TaskCache)
@flow.requires({"input1":get_data1, "input2":get_data2})
def use_data(task):
    data = task.inputLoad()
    df1 = data['input1']
    df2 = data['input2']
    df3 = df1.join(df2, lsuffix='1', rsuffix='2')
    df3['b']=df3['a1']*task.multiplier # use task parameter
    task.save(df3)

## Execute workflow

In [26]:
flow.add_params({'multiplier': d6tflow.IntParameter(default=0)})
use_params = {'multiplier':3}

In [33]:
flow.preview(use_data, params=use_params)

AttributeError: 'Flow' object has no attribute 'preview'

In [28]:
flow.run(use_data, params=use_params, forced_all_upstream=True, confirm=False)


===== Luigi Execution Summary =====

Scheduled 3 tasks of which:
* 3 ran successfully:
    - 1 get_data1(multiplier=3)
    - 1 get_data2(multiplier=3)
    - 1 use_data(multiplier=3)

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



In [30]:
flow.outputLoad(get_data1)

KeyError: 'get_data1'

In [32]:
flow.outputLoad(use_data)

,a1,a2,b
0,0,0,0
1,1,1,3
2,2,2,6
